In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime
import io
import datetime
import matplotlib.lines as mlines
import statsmodels.formula.api as smf
import datetime

In [ ]:
df = pd.read_csv('final_project_data.csv', parse_dates = True, index_col = 0)
df.head()

In [ ]:
#Lets check the missings
import missingno as msno
msno.matrix(df)

In [ ]:
#we find the first value of CPI in order the delet the NaN values that are before that date
initial_date = df['UK YoY CPI'].first_valid_index() 
#.fist_valid_index() returns the first non-NaN value
initial_date

In [6]:
#im gonna transform initial_date to first day of the last month 
initial_date = initial_date.replace(day=1)
#We reduce 1 month to this date
initial_date = initial_date - pd.DateOffset(months=1)
initial_date

Timestamp('2003-05-01 00:00:00')

In [7]:
#we are gonna drop all the values before initial_date
df = df[initial_date:]
df.head()

,USD2YB,UK2YB,UK YoY CPI,US YoY CPI,US Industrial Index,UK industrial Index
Date,,,,,,
2003-05-01,1.469,3.572,NaN,NaN,NaN,NaN
2003-05-02,1.565,3.631,NaN,NaN,NaN,NaN
2003-05-05,1.533,NaN,NaN,NaN,NaN,NaN
2003-05-06,1.428,3.644,NaN,NaN,NaN,NaN
2003-05-07,1.412,3.585,NaN,NaN,NaN,NaN


In [9]:
nan_values = df.isnull().sum()
print("NaN values in each column:")
print(nan_values)



NaN values in each column:
USD2YB                  299
UK2YB                   246
UK YoY CPI             5221
US YoY CPI             5220
US Industrial Index    5219
UK industrial Index    5220
dtype: int64


In [17]:
# Convert the index to datetime
df.index = pd.to_datetime(df.index)

# Resample the data to monthly frequency and take the mean of each month
df_monthly = df.resample('M').mean()

# Display the resulting monthly data
df_monthly

,USD2YB,UK2YB,UK YoY CPI,US YoY CPI,US Industrial Index,UK industrial Index
Date,,,,,,
2003-05-31,1.389429,3.537800,NaN,2.2,90.4281,101.1
2003-06-30,1.207667,3.417429,1.2,2.1,90.4073,99.8
2003-07-31,1.457773,3.560870,1.1,2.1,90.5218,100.4
2003-08-31,1.820333,4.043400,1.3,2.1,90.9896,101.0
2003-09-30,1.680381,4.156227,1.4,2.2,90.7858,101.2
...,...,...,...,...,...,...
2023-09-30,5.045500,4.988143,6.7,3.7,103.2411,95.0
2023-10-31,5.083095,4.851818,6.7,3.7,103.3346,94.9
2023-11-30,4.893381,4.618364,4.6,3.2,102.4592,93.7


In [19]:
# Specify the columns to be shifted
columns_to_shift = ['UK YoY CPI', 'US YoY CPI', 'US Industrial Index', 'UK industrial Index']

# Shift the specified columns using .shift(-1)
df_monthly[columns_to_shift] = df_monthly[columns_to_shift].shift(-1)

# Display the resulting DataFrame with shifted columns
df_monthly


,USD2YB,UK2YB,UK YoY CPI,US YoY CPI,US Industrial Index,UK industrial Index
Date,,,,,,
2003-05-31,1.389429,3.537800,1.1,2.1,90.5218,100.4
2003-06-30,1.207667,3.417429,1.3,2.1,90.9896,101.0
2003-07-31,1.457773,3.560870,1.4,2.2,90.7858,101.2
2003-08-31,1.820333,4.043400,1.4,2.3,91.3829,101.4
2003-09-30,1.680381,4.156227,1.4,2.0,91.5005,103.2
...,...,...,...,...,...,...
2023-09-30,5.045500,4.988143,4.6,3.2,102.4592,93.7
2023-10-31,5.083095,4.851818,3.9,3.1,102.4301,93.9
2023-11-30,4.893381,4.618364,NaN,NaN,102.4840,NaN


In [20]:
df_monthly.isnull().sum()


USD2YB                 1
UK2YB                  1
UK YoY CPI             3
US YoY CPI             3
US Industrial Index    2
UK industrial Index    3
dtype: int64

In [22]:
#handling missing values with interpolation
df_monthly.interpolate(method='linear', inplace= True)
df_monthly

,USD2YB,UK2YB,UK YoY CPI,US YoY CPI,US Industrial Index,UK industrial Index
Date,,,,,,
2003-05-31,1.389429,3.537800,1.1,2.1,90.5218,100.4
2003-06-30,1.207667,3.417429,1.3,2.1,90.9896,101.0
2003-07-31,1.457773,3.560870,1.4,2.2,90.7858,101.2
2003-08-31,1.820333,4.043400,1.4,2.3,91.3829,101.4
2003-09-30,1.680381,4.156227,1.4,2.0,91.5005,103.2
...,...,...,...,...,...,...
2023-09-30,5.045500,4.988143,4.6,3.2,102.4592,93.7
2023-10-31,5.083095,4.851818,3.9,3.1,102.4301,93.9
2023-11-30,4.893381,4.618364,3.9,3.1,102.4840,93.9


In [23]:
df_monthly.isnull().sum()

USD2YB                 0
UK2YB                  0
UK YoY CPI             0
US YoY CPI             0
US Industrial Index    0
UK industrial Index    0
dtype: int64

In [25]:
#handling missing values with interpolation
df_monthly.fillna(method='ffill', inplace=True)
df_monthly


,USD2YB,UK2YB,UK YoY CPI,US YoY CPI,US Industrial Index,UK industrial Index
Date,,,,,,
2003-05-31,1.389429,3.537800,1.1,2.1,90.5218,100.4
2003-06-30,1.207667,3.417429,1.3,2.1,90.9896,101.0
2003-07-31,1.457773,3.560870,1.4,2.2,90.7858,101.2
2003-08-31,1.820333,4.043400,1.4,2.3,91.3829,101.4
2003-09-30,1.680381,4.156227,1.4,2.0,91.5005,103.2
...,...,...,...,...,...,...
2023-09-30,5.045500,4.988143,4.6,3.2,102.4592,93.7
2023-10-31,5.083095,4.851818,3.9,3.1,102.4301,93.9
2023-11-30,4.893381,4.618364,3.9,3.1,102.4840,93.9


In [26]:
df_monthly.isnull().sum()

USD2YB                 0
UK2YB                  0
UK YoY CPI             0
US YoY CPI             0
US Industrial Index    0
UK industrial Index    0
dtype: int64